In [13]:
import cv2
import os
import xml.etree.ElementTree as ET

In [14]:
LABELS = ['AUDIO', 'AUDIO_INPUT', 'AUDIO_OUTPUT', 'DISPLAYPORT', 'DVI', 'ETHERNET', 'HDMI', 'PARALLEL', 'POWER', 'PS', 'PS2', 'USB', 'USBC', 'VGA']

In [15]:
# Function to parse bounding boxes from XML file
def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    for member in root.findall('object'):
        label = member[0].text
        bndbox = member[4]
        xmin = int(bndbox[0].text)
        ymin = int(bndbox[1].text)
        xmax = int(bndbox[2].text)
        ymax = int(bndbox[3].text)
        boxes.append({'label': label, 'xmin': xmin, 'ymin': ymin, 'xmax': xmax, 'ymax': ymax})
    return boxes

# Function to draw bounding boxes and labels on image
def draw_boxes_and_labels(img_name, input_path, output_path):
    # Load the image
    img = cv2.imread(os.path.join(input_path, img_name))

    boxes = parse_xml(os.path.join(input_path, img_name.split('.')[0] + '.xml'))
    
    # Iterate over each bounding box and label
    for box in boxes:
        x1, y1, x2, y2 = box['xmin'], box['ymin'], box['xmax'], box['ymax']
        
        # Draw the bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green color for bounding box

        # Draw the label text
        label_name = box['label']
        cv2.putText(img, label_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Blue color for text
    
    # Save the processed image
    cv2.imwrite(os.path.join(output_path, img_name), img)

# Define your input and output directories
input_image_dir = './content/image' 
output_dir = './content/output'

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the directory
for img_name in os.listdir(input_image_dir):
    if img_name.endswith('.jpg') or img_name.endswith('.jpeg'):
        draw_boxes_and_labels(img_name, input_image_dir, output_dir)